In [8]:
import tensorflow as tf
# tf.enable_eager_execution()
import numpy as np
import datetime
import os
import argparse
import matplotlib.pyplot as plt
import pandas as pd
import json
import glob
import tqdm
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers, models
from utils import *
tf.__version__

'1.15.0'

In [9]:
data_info = {
   "train_unlabel": 0, 
    "train_label": 0, 
    "validation": 0, 
    "test": 0
}
labels = ['DoS', 'Fuzzy', 'gear', 'RPM', 'Normal']
unknown_attack = ''
num_normal = 0
num_attack = 0
for f in ['./Data/{}/datainfo.txt'.format(l) for l in labels if l is not unknown_attack]:
    data_read = json.load(open(f))
    for key in data_info.keys():
        data_info[key] += data_read[key]
    if 'Normal' in f:
        num_normal += data_read['train_label']
    else:
        num_attack += data_read['train_label']
        
attack = 'all' # DoS, Fuzzy, gear, RPM, all
if unknown_attack != '':
    results_path = './Results/unknown/{}'.format(unknown_attack)
else:
    results_path = './Results/{}/'.format(attack)
print('Data info: ', data_info)
print('Num labeled normal: ', num_normal)
print('Num labeled attack: ', num_attack)

Data info:  {'train_unlabel': 359956, 'train_label': 39993, 'validation': 85701, 'test': 85701}
Num labeled normal:  24693
Num labeled attack:  15300


In [10]:
train_unlabel_paths = ['./Data/{}/train_unlabel'.format(l) for l in labels if l is not unknown_attack]
train_label_paths = ['./Data/{}/train_label'.format(l) for l in labels if l is not unknown_attack]
val_paths = ['./Data/{}/val'.format(l) for l in labels if l is not unknown_attack]
test_paths = ['./Data/{}/test'.format(l) for l in labels if l is not unknown_attack]

train_label = data_from_tfrecord(train_label_paths, data_info['train_label'], 1)
validation = data_from_tfrecord(val_paths,  data_info['validation'], 1)
test = data_from_tfrecord(test_paths, data_info['test'], 1)

print('Train unlabel: ', train_label_paths)
print('Train label: ', train_label_paths)
print('Validation: ', val_paths)

Train unlabel:  ['./Data/DoS/train_label', './Data/Fuzzy/train_label', './Data/gear/train_label', './Data/RPM/train_label', './Data/Normal/train_label']
Train label:  ['./Data/DoS/train_label', './Data/Fuzzy/train_label', './Data/gear/train_label', './Data/RPM/train_label', './Data/Normal/train_label']
Validation:  ['./Data/DoS/val', './Data/Fuzzy/val', './Data/gear/val', './Data/RPM/val', './Data/Normal/val']


In [11]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    X_train, y_train = data_stream(train_label, sess)
    X_val, y_val = data_stream(validation, sess)
    X_test, y_test = data_stream(test, sess)
    y_train = np.argmax(y_train, axis=1)
    y_val = np.argmax(y_val, axis=1)
    y_test = np.argmax(y_test, axis=1)
    print('Train: ', X_train.shape, y_train.shape)
    print('Validation: ', X_val.shape, y_train.shape)
    print('Test: ', X_test.shape, y_test.shape)

Train:  (39993, 841) (39993,)
Validation:  (85701, 841) (39993,)
Test:  (85701, 841) (85701,)


In [12]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes = (1000, 1000), random_state=1, max_iter=500).fit(X_train, y_train)

/home/thiennu/miniconda3/envs/Tensorflow1.x/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:619: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [13]:
y_pred = clf.predict(X_test)

KeyboardInterrupt: 

In [ ]:
evaluate(y_test, y_pred)

In [21]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(1000, input_dim=841, activation='relu'),
  tf.keras.layers.Dense(1000, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [22]:
model.compile(optimizer='adam',
              loss= 'binary_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [25]:
model.fit(X_train, y_train, epochs=30, batch_size=100, validation_data=(X_val, y_val))

Train on 39993 samples, validate on 85701 samples
Epoch 1/30
39993/39993 [==============================] - 2s 50us/sample - loss: 2.4991e-10 - acc: 1.0000 - val_loss: 0.0965 - val_acc: 0.9923
Epoch 2/30
39993/39993 [==============================] - 2s 49us/sample - loss: 2.3862e-10 - acc: 1.0000 - val_loss: 0.0971 - val_acc: 0.9923
Epoch 3/30
39993/39993 [==============================] - 2s 49us/sample - loss: 2.2996e-10 - acc: 1.0000 - val_loss: 0.0978 - val_acc: 0.9922
Epoch 4/30
39993/39993 [==============================] - 2s 50us/sample - loss: 2.2470e-10 - acc: 1.0000 - val_loss: 0.0982 - val_acc: 0.9922
Epoch 5/30
39993/39993 [==============================] - 2s 50us/sample - loss: 2.1797e-10 - acc: 1.0000 - val_loss: 0.0992 - val_acc: 0.9922
Epoch 6/30
39993/39993 [==============================] - 2s 50us/sample - loss: 2.1351e-10 - acc: 1.0000 - val_loss: 0.0996 - val_acc: 0.9922
Epoch 7/30
39993/39993 [==============================] - 2s 50us/sample - loss: 2.1087e-10 

In [36]:
y_pred = model.predict(X_test).flatten().astype(int)
evaluate(y_test, y_pred)

30956 1835
1 52909
False negative rate:  0.0559604769601415
Error rate:  0.021423320614695276
Precision:  0.9999676971282747
Recall:  0.9440395230398585
F1 score:  0.9711990964422414


In [8]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(29, 29, 1))) #[29, 29, 1] -> [29, 29, 32]
model.add(layers.MaxPooling2D((2, 2))) #[29, 29, 32] -> [14, 14, 32]
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(2, activation='relu'))
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 29, 29, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________

In [23]:
X_train = np.reshape(X_train, (X_train.shape[0], 29, 29, 1))
X_val = np.reshape(X_val, (X_val.shape[0], 29, 29, 1))
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=64, 
                    validation_data=(X_val, y_val))

Train on 39993 samples, validate on 85701 samples
Epoch 1/10
39993/39993 [==============================] - 5s 119us/sample - loss: 8.6545e-04 - acc: 0.9997 - val_loss: 0.0092 - val_acc: 0.9988
Epoch 2/10
39993/39993 [==============================] - 5s 116us/sample - loss: 1.1196e-04 - acc: 1.0000 - val_loss: 0.0133 - val_acc: 0.9985
Epoch 3/10
39993/39993 [==============================] - 5s 115us/sample - loss: 6.2849e-04 - acc: 0.9999 - val_loss: 0.0552 - val_acc: 0.9922
Epoch 4/10
39993/39993 [==============================] - 5s 120us/sample - loss: 0.0014 - acc: 0.9996 - val_loss: 0.0129 - val_acc: 0.9983
Epoch 5/10
39993/39993 [==============================] - 5s 126us/sample - loss: 6.0092e-04 - acc: 0.9998 - val_loss: 0.0083 - val_acc: 0.9987
Epoch 6/10
39993/39993 [==============================] - 5s 116us/sample - loss: 1.0856e-04 - acc: 0.9999 - val_loss: 0.0215 - val_acc: 0.9980
Epoch 7/10
39993/39993 [==============================] - 5s 113us/sample - loss: 4.8018e-

In [26]:
X_test = np.reshape(X_test, (X_test.shape[0], 29, 29, 1))
# y_test = np.argmax(y_test, axis=1)
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

In [27]:
evaluate(y_test, y_pred)

32650 141
6 52904
False negative rate:  0.00429996035497545
Error rate:  0.0017152658662092624
Precision:  0.9998162665360117
Recall:  0.9957000396450245
F1 score:  0.9977539077421425
